## Environment Setup

Place your conda yaml in the below section

In [5]:
name: xgb-customer-churn
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - python=3.6
  - xgboost
  - pip
  - pip:
      - mlflow>=1.6.0
      - matplotlib
      - boto3
      - smdebug

SyntaxError: invalid syntax (<ipython-input-5-073e4fd49224>, line 2)

In [ ]:
tracking_uri = "http://107.22.99.180/"
max_depth = 5
eta = 0.2
gamma = 4
min_child_weight = 6
eta = 0.2
subsample = 0.8
silent = 0
objective = "binary:logistic"
num_round = 200

## Place Imports 

Place all the imports that you are aware of here.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import boto3
import re


import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig
from sagemaker.model_monitor import DataCaptureConfig, DatasetFormat, DefaultModelMonitor
from sagemaker.s3 import S3Uploader, S3Downloader

## Utility Functions

Place the standalone functions in the below section

In [ ]:
def fetch_data(local_data_path):
    data = pd.read_csv(local_data_path)
    pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
    pd.set_option('display.max_rows', 10)         # Keep the output on one page
    return data

In [ ]:
def create_smdebug_hook(out_dir, train_data=None, validation_data=None, frequency=1, collections=None,):

    save_config = SaveConfig(save_interval=frequency)
    hook = Hook(
        out_dir=out_dir,
        train_data=train_data,
        validation_data=validation_data,
        save_config=save_config,
        include_collections=collections,
    )

    return hook

In [ ]:
def model_fn(model_dir):
    """Load a model. For XGBoost Framework, a default function to load a model is not provided.
    Users should provide customized model_fn() in script.
    Args:
        model_dir: a directory where model is saved.
    Returns:
        A XGBoost model.
        XGBoost model format type.
    """
    model_files = (file for file in os.listdir(model_dir) if os.path.isfile(os.path.join(model_dir, file)))
    model_file = next(model_files)
    try:
        booster = pickle.load(open(os.path.join(model_dir, model_file), 'rb'))
        format = 'pkl_format'
    except Exception as exp_pkl:
        try:
            booster = xgboost.Booster()
            booster.load_model(os.path.join(model_dir, model_file))
            format = 'xgb_format'
        except Exception as exp_xgb:
            print("Unable to load model: {} {}".format(str(exp_pkl), str(exp_xgb)))
            import sys
            sys.exit(-1)
    booster.set_param('nthread', 1)
    return booster, format

In [2]:
def parse_args():

    parser = argparse.ArgumentParser()

    parser.add_argument("--max-depth", type=int, default=5)
    parser.add_argument("--eta", type=float, default=0.2)
    parser.add_argument("--gamma", type=int, default=4)
    parser.add_argument("--min-child-weight", type=int, default=6)
    parser.add_argument("--subsample", type=float, default=0.8)
    parser.add_argument("--silent", type=int, default=0)
    parser.add_argument("--objective", type=str, default="binary:logistic")
    parser.add_argument("--num-round", type=int, default=50)
    parser.add_argument("--smdebug-path", type=str, default=None)
    parser.add_argument("--smdebug-frequency", type=int, default=1)
    parser.add_argument("--smdebug-collections", type=str, default='metrics')
    parser.add_argument("--output-uri", type=str, default="/opt/ml/output/tensors",
                        help="S3 URI of the bucket where tensor data will be stored.")

    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR', '/opt/ml/model'))

    print('\n-------------- Environment Variables -------------\n')
    for key, value in os.environ.items():
        print('{}={}'.format(key, value))

    args = parser.parse_args()

    return args

## Data Cleaning

In [3]:
def preprocessing(input_path):
    pass

## Visualize total data

Place all of the plots in the following section pertaining to the data

In [4]:
plot.show()

NameError: name 'plot' is not defined

## Training Code

Place training code here

In [ ]:
def run_training():
    args = parse_args()
    # enable auto logging
    mlflow.xgboost.autolog()
    train, validation = args.train, args.validation
    parse_csv = "?format=csv&label_column=0"
    dtrain = xgboost.DMatrix(train+parse_csv)
    dval = xgboost.DMatrix(validation+parse_csv)

    # enable auto logging
    mlflow.xgboost.autolog()

    watchlist = [(dtrain, "train"), (dval, "validation")]
    with mlflow.start_run():
        params = {
            "max_depth": args.max_depth,
            "eta": args.eta,
            "gamma": args.gamma,
            "min_child_weight": args.min_child_weight,
            "subsample": args.subsample,
            "silent": args.silent,
            "objective": args.objective}

        # The output_uri is a the URI for the s3 bucket where the metrics will be
        # saved.
        output_uri = (
            args.smdebug_path
            if args.smdebug_path is not None
            else args.output_uri
        )

        collections = (
            args.smdebug_collections.split(',')
            if args.smdebug_collections is not None
            else None
        )

        hook = create_smdebug_hook(
            out_dir=output_uri,
            frequency=args.smdebug_frequency,
            collections=collections,
            train_data=dtrain,
            validation_data=dval,
        )

        model = xgboost.train(
            params=params,
            dtrain=dtrain,
            evals=watchlist,
            num_boost_round=args.num_round,
            callbacks=[hook])

        if not os.path.exists(args.model_dir):
            os.makedirs(args.model_dir)

        model_location = os.path.join(args.model_dir, 'xgboost-model')
        pickle.dump(model, open(model_location, 'wb'))

        mlflow.xgboost.log_model(model, 'model', registered_model_name='xgb-customer-churn')

## Run Training

In [ ]:
warnings.filterwarnings("ignore")
np.random.seed(40)
try:
    run_training()
except Exception as e:
    logger.exception("Unable to execute training", e)

In [36]:
import sagemaker
import boto3
import time
from sagemaker.predictor import csv_serializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV

import sagemaker
from sagemaker import get_execution_role
sess = sagemaker.Session()
role = get_execution_role()
endpoint_name = 'dev-xgb-customer-churn-v2'


predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=csv_serializer,
                                content_type=CONTENT_TYPE_CSV)

column_names = "Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,Intl Calls,CustServ Calls,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes\n"

with open('test-data/test_sample.csv', 'r') as f:
    for row in f:
        payload_orig = row.rstrip('\n')
        payload = row.rstrip('\n')
        payload = column_names + payload
        response = predictor.predict(data=payload)
        print(response)
        time.sleep(0.5)

b'[0.011914667673408985]'
b'[0.008735105395317078]'
b'[0.15768414735794067]'
b'[0.008569112047553062]'
b'[0.023335663601756096]'
b'[0.6892004609107971]'


KeyboardInterrupt: 

In [37]:
!head 'test-data/test_sample.csv'

132,25,113.2,96,269.9,107,229.1,87,7.1,7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
112,17,183.2,95,252.8,125,156.7,95,9.7,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
91,24,93.5,112,183.4,128,240.7,133,9.9,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1
22,0,110.3,107,166.5,93,202.3,96,9.5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
102,0,186.8,92,173.7,123,250.9,131,9.7,4,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
118,21,156.5,122,209.2,125,158.7,81,11.1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1
178,35,175.4,88,190.0,65,138.7,94,10.5,3,2,0,0

![CodePipeline](codepipeline.png "Sample CodePipeline View")

![CodePipeline](codepipeline.png "Sample CodePipeline View")